In [ ]:
import requests
import pandas as pd
from pyspark.sql import SparkSession
from bs4 import BeautifulSoup, NavigableString, Tag

URL = "https://thepurplepigchicago.com/drink"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
menu = soup.find_all("div", class_="menu-section")

In [ ]:
def get_wines_df():
    sections = []
    items = []
    staging = []
    def clean_extra(extra):
        if extra == None:
            return ''
        else:
            return ' '.join(x.strip() for x in extra.replace('\n','').split('/'))

    for i in menu:
        sections.append((i.find('div',{'class':'menu-section-title'}).text, \
                             zip([x.text.split('|')[1] if '|' in x.text else x.text for x in i.find_all('div',{'class':'menu-item-title'})], \
                            [x.text.replace('|','') for x in i.find_all("div", class_="menu-item-description") if x != None], \
                            [clean_extra(x.text) for x in i.find_all("div", class_="menu-item-price-bottom")])))



    for i in sections:
        category = i[0]
        for items in i[1]:
            staging.append(["PurplePig"]+[category] + [x for x in items])

    dict_holder = []
    key_list = ['origin','section', 'name', 'description','extra']

    for i in staging:
        dict_from_list = dict(zip(key_list, i))
        dict_holder.append(dict_from_list)
    return(pd.DataFrame(dict_holder))
    

In [ ]:
df.to_parquet('Wines.parquet')

In [ ]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.7.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [ ]:
df = spark.read.parquet('Wines.parquet')

In [ ]:
df.write.format('delta').save('tmp/delta/')